# Criando um Perceptron de Multíplas Camadas

Este notebook corresponde a atividade prática da seção 3.4 no livro do curso. Nesta atividade você aprenderá a desenvolver o Multilayer Perceptron (MLP) e aplica-lo para resolver problemas não linearmente separáveis.

## 1 - Pacotes

Execute o bloco abaixo para importar os pacotes necessarios. 

- [tensorflow](https://www.tensorflow.org/) um framework para machine learning
- [numpy](www.numpy.org) pacote de bilbiotecas para computação científica.
- [matplotlib](http://matplotlib.org) biblioteca para desenho de gráficos.
- [sklearn.datasets](http://scikit-learn.org/stable/datasets/index.html) pacote do scikit-learn para geração de datasets artificiais

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets

#código local com a função que desenha os separadores
from util import draw_separator


## 2 - Gerando um dataset artificial

As funções "make" do módulo datasets do scikit-learn produzem distribuições artificiais que podem ser usados para testar modelos de aprendizado de máquina.

In [ ]:
#Setando o seed do tensorflow e numpy com 0 para gerar uma sequencia aleatoria conhecida
tf.set_random_seed(0)
np.random.seed(0)

#número de classes do problema
num_classes = 2

#gerando o dataset (descomente uma das funções abaixo para gerar um dataset)

#dataset_X, dataset_Y = sklearn.datasets.make_moons(200, noise=0.20)
#dataset_X, dataset_Y = sklearn.datasets.make_circles(200, noise=0.05)
#dataset_X, dataset_Y = sklearn.datasets.make_gaussian_quantiles(n_features=2, n_classes=num_classes)

#plotando o dataset
_ = plt.scatter(dataset_X[:,0], dataset_X[:,1], s=40, c=dataset_Y, cmap=plt.cm.Spectral)



## 3 - Construindo o modelo MLP

A função abaixo cria uma arquitura de rede neural do tipo MLP.

In [ ]:
#a função recebe como parametro a quantidade de features e número de classes no problema
def build_net(n_features, n_classes):
    
    #Por praticidade os tensores são salvos neste dicionário
    Dic = {}
    
    # ---- INICIO DA REDE NEURAL ----
    
    # Placeholders (X (entrada) e Y (saída) da rede)
    placeholder_X = tf.placeholder(dtype=tf.float32, shape=[None, n_features])
    Dic["placeholder_X"] = placeholder_X
    
    placeholder_Y = tf.placeholder(dtype=tf.int64, shape=[None])
    Dic["placeholder_Y"] = placeholder_Y

    # camada escondida
    hidden_layer1 = tf.layers.dense(placeholder_X, 10, activation=tf.nn.relu)
    Dic["layer1"] = hidden_layer1
    
    #camada de saída
    out = tf.layers.dense(hidden_layer1, n_classes, name="output")
     
    # ---- FIM DA REDE NEURAL ----
    
    #adaptando o Label Y para o modelo One-Hot Label
    one_hot = tf.one_hot(placeholder_Y, depth=n_classes)
    
    # Função de perda/custo/erro
    loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot, logits=out) 
    Dic["loss"] = loss
    
    # Otimizador
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.07).minimize(loss)
    Dic["opt"] = opt
    
    #Softmax
    softmax = tf.nn.softmax(out)
    Dic["softmax"] = softmax
    
    #Classe
    class_ = tf.argmax(softmax,1)
    Dic["class"] = class_
    
    #Acurácia
    compare_prediction = tf.equal(class_, placeholder_Y)
    accuracy = tf.reduce_mean(tf.cast(compare_prediction, tf.float32))
    Dic["accuracy"] = accuracy
    
    return Dic

## 4 - Iniciando o Tensorflow

O código abaixo inicia uma sessão no TensorFlow. E em segudia, carrega o gráfo de computação definido na função "build_net".

In [ ]:
#Iniciando
sess = tf.InteractiveSession()

#obtendo o número de features 
n_features = dataset_X.shape[1]

#construindo o modelo de rede
Dic_cg = build_net(n_features,num_classes)

#inicializando as variveis do tensorflow
sess.run(tf.global_variables_initializer())

## 6 - Treinamento da rede

Nessa étapa o modelo instanciado é treinado com os dados do dataset. O treinamento ocorre em um loop que é executado 1000 vezes (1000 épocas). A cada 100 épocas o erro é calculado e impresso. Por fim, ao termino do treinamento, é calculada a acurácia do modelo.

In [ ]:
#definindo o número de épocas
epochs = 2000

for i in range(epochs):
    
    sess.run(Dic_cg["opt"], feed_dict={Dic_cg["placeholder_X"]: dataset_X, Dic_cg["placeholder_Y"]: dataset_Y})
    
    # a cada 100 épocas o erro é impresso
    if  i % 100 == 0:
        erro_train = sess.run(Dic_cg["loss"], feed_dict={Dic_cg["placeholder_X"]: dataset_X, Dic_cg["placeholder_Y"]: dataset_Y})
        print("O erro na época", i,"é", erro_train)
        
#após o fim do treino, é calculada a acurácia
acc = sess.run(Dic_cg["accuracy"], feed_dict={Dic_cg["placeholder_X"]: dataset_X, Dic_cg["placeholder_Y"]: dataset_Y})
print("A accurácia é:", acc)



## 6 - Realizando predições

Ao executar o tensor "class", passando um exemplo como parâmetro é possivel obter a predição correspondente.

In [ ]:
cla = sess.run(Dic_cg["class"], feed_dict={Dic_cg["placeholder_X"]: dataset_X[:1]})

print("A Classe do ponto", dataset_X[:1], " é:", cla)

## 6 - Visualizando o separador

Com a função "draw_separator" é possível visualizar como o modelo separada os dados de entrada.

In [ ]:

#desenhando o separador
draw_separator(dataset_X, dataset_Y, sess, Dic_cg["placeholder_X"], Dic_cg["class"])
